<a href="https://colab.research.google.com/github/shazhao57/Retrieve-Emails-Llama-index/blob/main/Vector_Index_Retrieve_office_account_email.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pip Installation

In [1]:
!pip install llama-index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.4/544.4 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 58.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 5.2 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.6.3
    Uninstalling typing_extensions-4.6.3:
      Successfully uninstalled typing_extensions-4.6.3


In [2]:
!pip install google-cloud-secret-manager

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.7/116.7 kB 5.6 MB/s eta 0:00:00


In [3]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 58.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.3 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=18608876a7e3c3f7235449325fd2ab1d78366e9a6766256ba2ddd129f47e3967
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [4]:
!pip install InstructorEmbedding

# Change ada into instructor

In [5]:
# change ada into instructor
from langchain.embeddings import HuggingFaceInstructEmbeddings
from llama_index import LangchainEmbedding

#config model
model_name = "hkunlp/instructor-xl"
encode_kwargs = {'normalize_embeddings': True,
                 "show_progress_bar":True}

# Link to info: https://huggingface.co/hkunlp/instructor-xl (actual implementation is a little complicated which is why we use langchain wrapper)
# Make langchain embed model object

hf = HuggingFaceInstructEmbeddings(
    model_name=model_name,
    encode_kwargs=encode_kwargs
)

#Make model that works for llama_index
embed_model=LangchainEmbedding(hf)


load INSTRUCTOR_Transformer
max_seq_length  512


# Retrieve emails

In [6]:
from google.colab import auth
from google.colab import drive

auth.authenticate_user()

In [7]:
from google.cloud import secretmanager_v1 as secretmanager
from google.auth import default as google_auth

import googleapiclient.discovery
import googleapiclient.errors

In [8]:
"""Google Mail reader."""
import email
from typing import Any, List
from llama_index.readers.base import BaseReader
from llama_index.readers.schema.base import Document
from pydantic import BaseModel
import base64
from datetime import datetime
import pytz

SCOPES = ["https://www.googleapis.com/auth/gmail.readonly"]


class GmailReader(BaseReader, BaseModel):

    query: str = None
    use_iterative_parser: bool = False
    service: Any


    def load_data(
        self
    ) -> List[Document]:
        """Load emails from the user's account
        """
        from googleapiclient.discovery import build
        import json
        from google.oauth2.credentials import Credentials

        credentials = self._get_credentials()
        #credentials is a JSON string
        credentials_json = credentials
        # Load the JSON string into a dictionary
        credentials_dict = json.loads(credentials_json)
        #Create Google API credentials object
        credentials1 = Credentials.from_authorized_user_info(credentials_dict)

        if not self.service:
            self.service = build('gmail', 'v1', credentials=credentials1)

        messsages = self.search_messages()

        results = []
        for message in messsages:
            text = message.pop('body')
            extra_info = message
            results.append(Document(text=text, metadata=extra_info))

        return results

    def _get_credentials(self) -> Any:

        secret_name = 'tanaka-office-gmail-credencial'
        project_id = 'vivid-kite-315522'

        # Create the Secret Manager client
        client = secretmanager.SecretManagerServiceClient()
        # Build the secret name path
        secret_path = f"projects/{project_id}/secrets/{secret_name}/versions/latest"
        # Access the secret version
        response = client.access_secret_version(request={"name": secret_path})
        # Retrieve the secret value
        secret_value = response.payload.data.decode("UTF-8")
        # Use the secret value as your credentials
        return secret_value

    def search_messages(self):
        query = self.query

        messages = self.service.users().messages().list(
            userId='me',
            q=query,
        ).execute().get('messages', [])

        result = []
        try:
            for message in messages:
                message_data = self.get_message_data(message)
                if not message_data:
                    continue
                result.append(message_data)
        except Exception as e:
            raise Exception("Can't get message data" + str(e))

        return result

    def get_message_data(self, message):
        message_id = message['id']
        message_data = self.service.users().messages().get(
            format="raw",
            userId='me',
            id=message_id).execute()

        # Convert the raw message data into a MIME message
        body_bytes = base64.urlsafe_b64decode(message_data['raw'].encode('ASCII'))
        mime_msg = email.message_from_bytes(body_bytes)

        if self.use_iterative_parser:
            body = self.extract_message_body_iterative(mime_msg)
        else:
            body = self.extract_message_body(mime_msg)

        if not body:
            return None

        # Extract the headers
        headers = dict(mime_msg.items())

        # decode the Subject header
        from email.header import decode_header

        # decode the Subject header
        decoded_header = decode_header(headers.get('Subject'))
        decoded_subject = ''
        for text, charset in decoded_header:
            if isinstance(text, bytes):
                decoded_subject += text.decode(charset if charset else 'utf-8')
            else:
                decoded_subject += text
        headers['Subject'] = decoded_subject

        # change the time zone to -7000
        email_date = datetime.strptime(headers.get('Date'), "%a, %d %b %Y %H:%M:%S %z")
        email_date = email_date.astimezone(pytz.timezone('Etc/GMT+7'))
        headers['Date'] = email_date.strftime("%a, %d %b %Y %H:%M:%S %z")

        return {
            "body": body,
            "Subject": headers.get('Subject'),
            "From": headers.get('From'),
            "To": headers.get('To'),
            "Date": headers.get('Date'),
        }

    def extract_message_body_iterative(self, mime_msg):
          body_text = ''

          if mime_msg.is_multipart():
              for part in mime_msg.get_payload():
                  if part.get_content_type() == 'text/plain':
                      charset = part.get_content_charset()
                      if charset is not None:
                          body_text += part.get_payload(decode=True).decode(str(charset), errors="replace")
                      else:
                          body_text += part.get_payload()
          elif mime_msg.get_content_type() == 'text/plain':
              charset = mime_msg.get_content_charset()
              if charset is not None:
                  body_text = mime_msg.get_payload(decode=True).decode(str(charset), errors="replace")
              else:
                  body_text = mime_msg.get_payload()

          return body_text

    def extract_message_body(self, mime_msg):
          body_text = ''

          if mime_msg.is_multipart():
              for part in mime_msg.get_payload():
                  # Check if the part is 'text/plain'
                  if part.get_content_type() == 'text/plain':
                      charset = part.get_content_charset()
                      # Check if the part is encoded in base64
                      if charset is not None:
                          body_text += part.get_payload(decode=True).decode(str(charset), errors="replace")
                      else:
                          body_text += part.get_payload()
          elif mime_msg.get_content_type() == 'text/plain':
              charset = mime_msg.get_content_charset()
              # Check if the message is encoded in base64
              if charset is not None:
                  body_text = mime_msg.get_payload(decode=True).decode(str(charset), errors="replace")
              else:
                  body_text = mime_msg.get_payload()

          return body_text


# Store emails into vector database

In [9]:
import openai
import os
import json

from google.cloud import secretmanager

# Set up the Secret Manager client
client1 = secretmanager.SecretManagerServiceClient()
name1 = "projects/339902708114/secrets/openai_api_key/versions/latest"
response1 = client1.access_secret_version(request={"name": name1})

# Retrieve the API key from the secret
api_key1 = response1.payload.data.decode("UTF-8")

# Set the OpenAI API key
openai.api_key = api_key1
os.environ["OPENAI_API_KEY"]=api_key1


In [49]:
#from langchain.embeddings.huggingface import HuggingFaceInstructEmbeddings
#from llama_index import LangchainEmbedding,ServiceContext
from llama_index.langchain_helpers.text_splitter import TokenTextSplitter

#reader = GmailReader(query="from:abjpd1@gmail.com")
reader = GmailReader(query="from:h.etzko@gmail.com")
#reader = GmailReader(query="from:me  after:2023-06-10")

documents = reader.load_data()
filter_documents = documents[0:1] + documents[2:] # the emails in the vector. documents[0:6]: from 0 to 5, documents[7:]: from 7 to the end.

new_documents = []
for document in filter_documents:
    text_splitter = TokenTextSplitter(separator=" ", chunk_size=160, chunk_overlap=130)
    text_chunks = text_splitter.split_text(document.text)
    for chunk in text_chunks:
        new_document = Document(text = chunk, metadata = document.metadata)
        new_documents.append(new_document)

print(len(documents))
print(len(filter_documents))
print(len(new_documents))

38
37
1603


In [ ]:
# vector database
from llama_index import VectorStoreIndex
index = VectorStoreIndex.from_documents(new_documents)

In [44]:
import pprint
import textwrap # read the output line by line. keep output stay in the view.
pprint.pprint(documents[1].metadata)
print("---------------------------------------------------------")
print('\n'.join(textwrap.wrap(documents[1].text, width=80))) # the email we need to answer

{'Date': 'Fri, 23 Jun 2023 13:00:50 -0700',
 'From': 'Henry Etzkowitz <h.etzko@gmail.com>',
 'Subject': 'Clubhouse access',
 'To': '"oakcreek@sares-regis.com" <oakcreek@sares-regis.com>, Firoozeh '
       'Dastmalchi\r\n'
       '\t<firoozehdh@gmail.com>, Marty Wasserman <deeperlook@aol.com>, '
       'Kristina\r\n'
       ' Loquist <Kristina.Loquist@bos.sccgov.org>, Jim Hersh '
       '<hershj@salve.edu>,\r\n'
       '\tRebecca Eisenberg <rebecca@rebecca4water.com>, Roberta Ahlquist\r\n'
       '\t<roberta.ahlquist@sjsu.edu>, Terry Beaubois <tbeaubois@gmail.com>, '
       'Bae Hyo\r\n'
       '\t<khbae@busan.go.kr>, Khonika Gope <khonika@stanford.edu>, Christiane '
       'Gebhardt\r\n'
       '\t<christiane.gebhardt@t-online.de>'}
---------------------------------------------------------
CAUTION: This email originated from outside of the organization. Be cautious of
opening attachments and clicking on links.  ________________________________
Does this mean that you are “barricading” 

# Retrieve relevant chunks


In [48]:
import pprint

retriever = index.as_retriever(similarity_top_k=10)
results = retriever.retrieve(documents[1].text)

for result in results:
    print("--------------------------------------------------------------------")
    #pprint.pprint(result.node.metadata)
    print('\n'.join(textwrap.wrap(result.node.text, width=80)))

--------------------------------------------------------------------
and facing potential reprimand (like the courageous Indiana abortion Physician),
Community of Oak Creek Residents (COCR) withdraws its request that she weigh in
on our behalf on the specific issues of clubhouse hours, illegal emplacement of
door and removal of billiard and other social activities in favor of
bureaucratic offices, shifting administrative activities from  an ancillary
support to a primary front and center site, with the apparent intent of
depriving  students post docs and other residents of the uplifting spacious site
architecturally intended for their sociability
--------------------------------------------------------------------
and newspapers on coffeehouse holders to bring residents together are now being
separated with the call of the newly hired management company to: “go to your
apartments.” Umbrellas on the deck tables have been removed as have the low
seating tables themselves, replaced by unp

In [15]:
query_engine = index.as_query_engine()
response = query_engine.query("What is the title of the recent email")

print(response)


Re: Valley Water Board Meeting 1 PM Tuesday June 26 - Please call in to support me?


# Try if openai works

In [ ]:
from langchain.llms import OpenAI
llm= OpenAI(model_name="gpt-3.5-turbo")
llm("What's 5 plus 4") #simple test query

# use RecursiveCharacterTextSplitter chunk up the email body into smaller chunks

In [ ]:
#RecursiveCharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

text = "The Recursive Text Splitter Module is a module in the LangChain library that can be used to split text recursively. This means that the module will try to split the text into different characters until the chunks are small enough."

splitter = RecursiveCharacterTextSplitter(chunk_size=30,chunk_overlap=8)

chunks = splitter.split_text(text)

for chunk in chunks:
    print(chunk)

The Recursive Text Splitter
Module is a module in the
in the LangChain library that
that can be used to split
split text recursively. This
This means that the module
module will try to split the
the text into different
characters until the chunks
chunks are small enough.


In [ ]:
'''
def chunk_string(s, n):
    """Split a string into chunks of size n."""
    return [s[i:i+n] for i in range(0, len(s), n)]

main:
    chunk_size = 1024  # adjust as needed
    new_documents = []

    for document in documents:
        text_chunks = chunk_string(document.text, chunk_size)

        for chunk in text_chunks:
            # Create a new Document with the chunk and the same extra_info
            new_document = Document(chunk, extra_info=document.extra_info)
            new_documents.append(new_document)
    print(len(documents))
    print(len(new_documents))
    print(new_documents[45].text)
'''